## Configuration Parameters

**Customize these parameters for your analysis:**

In [ ]:
# ============================================================================
# CONFIGURATION - CUSTOMIZE THESE PARAMETERS
# ============================================================================

# Station information
NETWORK = "AM"  # Network code
STATION = "RB38A"  # Station code to analyze (e.g., 'R4017', 'RB38A', 'R1796')
CHANNEL = "EHZ"  # Channel (vertical component)

# Time window for analysis
START_TIME = "2024-11-27T17:00:00"  # UTC time (YYYY-MM-DDTHH:MM:SS)
END_TIME = "2024-11-27T17:30:00"    # UTC time (30 minutes)

# Model parameters (should match training)
WINDOW_LENGTH_SEC = 5.0  # Window length in seconds
WINDOW_OVERLAP = 0.5  # 50% overlap

# Visualization parameters
CONFIDENCE_THRESHOLD = 0.5  # Minimum confidence for positive detection
N_EXAMPLE_WINDOWS = 12  # Number of example windows to display

print("✓ Configuration loaded")
print(f"  Target station: {NETWORK}.{STATION}")
print(f"  Time window: {START_TIME} to {END_TIME}")
print(f"  Window: {WINDOW_LENGTH_SEC}s with {WINDOW_OVERLAP*100:.0f}% overlap")
print(f"  Confidence threshold: {CONFIDENCE_THRESHOLD}")


✓ Configuration loaded
  Target station: AM.RB38A
  Window: 5.0s with 50% overlap
  Confidence threshold: 0.5


## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
import sys
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ObsPy imports
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# PyTorch imports
import torch
import torch.nn.functional as F

# Add parent directory to path
sys.path.insert(0, os.path.abspath('../..'))
from src.models.cnn import SeismicCNN, CompactSeismicCNN

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 10

print("\n✓ Libraries imported successfully!")


Using device: cpu

✓ Libraries imported successfully!


## Download Seismic Data

In [ ]:
# Initialize client
client = Client(base_url='https://data.raspberryshake.org')

# Set time window
start_time = UTCDateTime(START_TIME)
end_time = UTCDateTime(END_TIME)

print(f"Time window:")
print(f"  Start: {start_time}")
print(f"  End: {end_time}")
print(f"  Duration: {(end_time - start_time)/60:.1f} minutes")

# Download data
print(f"\nDownloading data for station {NETWORK}.{STATION}...")
try:
    stream = client.get_waveforms(
        network=NETWORK,
        station=STATION,
        location="00",
        channel=f"{CHANNEL[0:2]}*",
        starttime=start_time,
        endtime=end_time
    )
    
    # Preprocess
    stream.detrend('linear')
    stream.detrend('demean')
    stream.taper(max_percentage=0.05)
    
    # Select vertical component
    trace = stream.select(channel=CHANNEL)[0]
    sampling_rate = trace.stats.sampling_rate
    
    print(f"\n✓ Data downloaded successfully!")
    print(f"  Station: {trace.stats.station}")
    print(f"  Channel: {trace.stats.channel}")
    print(f"  Sampling rate: {sampling_rate} Hz")
    print(f"  Duration: {(trace.stats.endtime - trace.stats.starttime):.1f} seconds")
    print(f"  Samples: {len(trace.data)}")
    
except Exception as e:
    raise RuntimeError(f"Failed to download data: {e}")


Searching for earthquakes M>=4.0 in region...


FDSNNoDataException: No data available for request.
HTTP Status code: 204
Detailed response of server:



## Load Trained Model

In [ ]:
# Find most recent model
models_dir = Path("../../models")
model_files = sorted(models_dir.glob("seismic_cnn_*.pth"))

if not model_files:
    raise FileNotFoundError("No trained model found. Run ../03_training/train_cnn_multiclass.ipynb first.")

model_path = model_files[-1]
print(f"Loading model: {model_path.name}")

# Load model checkpoint
checkpoint = torch.load(model_path, map_location=device)

# Extract model configuration
model_type = checkpoint['model_type']
num_classes = checkpoint['num_classes']
input_channels = checkpoint['input_channels']
input_length = checkpoint['input_length']
class_names = checkpoint['class_names']
test_accuracy = checkpoint.get('test_accuracy', 0)

print(f"\nModel configuration:")
print(f"  Type: {model_type}")
print(f"  Classes: {class_names}")
print(f"  Input shape: ({input_channels}, {input_length})")
print(f"  Test accuracy: {test_accuracy*100:.2f}%")

# Initialize model
if model_type == 'standard':
    model = SeismicCNN(
        num_classes=num_classes,
        input_channels=input_channels,
        input_length=input_length
    )
else:
    model = CompactSeismicCNN(
        num_classes=num_classes,
        input_channels=input_channels,
        input_length=input_length
    )

# Load weights
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print(f"\n✓ Model loaded successfully!")

## Window Data and Make Predictions

In [ ]:
# Windowing parameters
window_samples = int(WINDOW_LENGTH_SEC * sampling_rate)
step_samples = int(window_samples * (1 - WINDOW_OVERLAP))

# Check if window size matches model
if window_samples != input_length:
    print(f"⚠️  Warning: Window size ({window_samples}) doesn't match model input ({input_length})")
    print(f"    Adjusting window size to match model...")
    window_samples = input_length
    WINDOW_LENGTH_SEC = window_samples / sampling_rate

# Calculate number of windows
n_samples = len(trace.data)
n_windows = (n_samples - window_samples) // step_samples + 1

print(f"Windowing configuration:")
print(f"  Window length: {WINDOW_LENGTH_SEC:.1f} seconds ({window_samples} samples)")
print(f"  Step size: {step_samples} samples")
print(f"  Total windows: {n_windows}")

# Extract windows and predict
predictions = []
probabilities = []
window_times = []
windowed_data = []

print(f"\nProcessing windows...")
with torch.no_grad():
    for i in range(n_windows):
        start_idx = i * step_samples
        end_idx = start_idx + window_samples
        
        if end_idx > n_samples:
            break
        
        # Extract window
        window = trace.data[start_idx:end_idx]
        
        # Normalize
        window_normalized = (window - np.mean(window)) / (np.std(window) + 1e-10)
        
        # Store for visualization
        windowed_data.append(window_normalized)
        
        # Prepare for model: (1, 1, window_length)
        window_tensor = torch.FloatTensor(window_normalized).unsqueeze(0).unsqueeze(0).to(device)
        
        # Predict
        output = model(window_tensor)
        probs = F.softmax(output, dim=1).cpu().numpy()[0]
        pred = np.argmax(probs)
        
        # Store results
        predictions.append(pred)
        probabilities.append(probs)
        window_time = trace.stats.starttime + (start_idx / sampling_rate)
        window_times.append(window_time)
        
        if (i + 1) % 100 == 0:
            print(f"  Processed {i + 1}/{n_windows} windows...")

# Convert to arrays
predictions = np.array(predictions)
probabilities = np.array(probabilities)
windowed_data = np.array(windowed_data)

print(f"\n✓ Predictions complete!")
print(f"  Total windows processed: {len(predictions)}")

# Calculate detection statistics
unique, counts = np.unique(predictions, return_counts=True)
print(f"\nDetection summary:")
for label, count in zip(unique, counts):
    print(f"  {class_names[label]}: {count} windows ({count/len(predictions)*100:.1f}%)")

## Visualize Classification Timeline

In [ ]:
# Create time axis in seconds from start
times_sec = np.array([(t - trace.stats.starttime) for t in window_times])

# Define colors for each class
colors_map = {0: 'gray', 1: 'orange', 2: 'red'}
colors_list = [colors_map[p] for p in predictions]

fig, axes = plt.subplots(4, 1, figsize=(16, 12))

# 1. Raw waveform
ax0 = axes[0]
time_axis = trace.times()
ax0.plot(time_axis, trace.data, 'k-', linewidth=0.5, alpha=0.7)
ax0.set_ylabel('Amplitude', fontsize=12)
ax0.set_title(f'Raw Seismogram - {NETWORK}.{STATION}.{CHANNEL}', fontsize=14)
ax0.grid(True, alpha=0.3)
ax0.set_xlim(0, time_axis[-1])

# 2. Classification timeline
ax1 = axes[1]
for i, (time, pred) in enumerate(zip(times_sec, predictions)):
    ax1.scatter(time, pred, c=colors_map[pred], s=50, alpha=0.6)
ax1.set_ylabel('Predicted Class', fontsize=12)
ax1.set_yticks([0, 1, 2])
ax1.set_yticklabels(class_names)
ax1.set_title('Classification Timeline', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, times_sec[-1])

# Add legend
legend_elements = [mpatches.Patch(color=colors_map[i], label=class_names[i]) for i in range(num_classes)]
ax1.legend(handles=legend_elements, loc='upper right')

# 3. Probability for Earthquake class
ax2 = axes[2]
ax2.fill_between(times_sec, 0, probabilities[:, 2], color='red', alpha=0.3, label='Earthquake')
ax2.plot(times_sec, probabilities[:, 2], 'r-', linewidth=1)
ax2.axhline(y=CONFIDENCE_THRESHOLD, color='k', linestyle='--', linewidth=1, label=f'Threshold ({CONFIDENCE_THRESHOLD})')
ax2.set_ylabel('Probability', fontsize=12)
ax2.set_title('Earthquake Detection Probability', fontsize=14)
ax2.set_ylim(0, 1)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, times_sec[-1])

# 4. Probability for all classes
ax3 = axes[3]
ax3.plot(times_sec, probabilities[:, 0], color='gray', linewidth=1, label='Noise', alpha=0.7)
ax3.plot(times_sec, probabilities[:, 1], color='orange', linewidth=1, label='Traffic', alpha=0.7)
ax3.plot(times_sec, probabilities[:, 2], color='red', linewidth=1, label='Earthquake', alpha=0.7)
ax3.set_ylabel('Probability', fontsize=12)
ax3.set_xlabel('Time (seconds)', fontsize=12)
ax3.set_title('All Class Probabilities', fontsize=14)
ax3.set_ylim(0, 1)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, times_sec[-1])

plt.tight_layout()
plt.show()

print("✓ Timeline visualization complete")

## Visualize Example Detections

In [ ]:
# Select example windows from each class
n_rows = 3
n_cols = 4
n_examples = min(N_EXAMPLE_WINDOWS, n_rows * n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 9))
axes = axes.flatten()

examples_per_class = n_examples // num_classes
plot_idx = 0

for class_idx in range(num_classes):
    # Find windows of this class with high confidence
    class_mask = predictions == class_idx
    class_probs = probabilities[class_mask, class_idx]
    class_indices = np.where(class_mask)[0]
    
    if len(class_indices) == 0:
        continue
    
    # Select highest confidence examples
    top_indices = np.argsort(class_probs)[-examples_per_class:]
    selected_indices = class_indices[top_indices]
    
    for idx in selected_indices:
        if plot_idx >= n_examples:
            break
            
        ax = axes[plot_idx]
        
        # Get window data
        waveform = windowed_data[idx]
        pred_label = predictions[idx]
        confidence = probabilities[idx, pred_label]
        
        # Plot waveform
        time_ax = np.arange(len(waveform)) / sampling_rate
        ax.plot(time_ax, waveform, color=colors_map[pred_label], linewidth=1)
        
        # Title with info
        title = f"{class_names[pred_label]}\n"
        title += f"Conf: {confidence*100:.1f}% | T={times_sec[idx]:.1f}s"
        ax.set_title(title, fontsize=9, color=colors_map[pred_label])
        ax.set_xlabel('Time (s)', fontsize=8)
        ax.set_ylabel('Amplitude', fontsize=8)
        ax.grid(True, alpha=0.3)
        ax.tick_params(labelsize=7)
        
        plot_idx += 1

# Hide unused subplots
for i in range(plot_idx, len(axes)):
    axes[i].axis('off')

plt.suptitle(f'Example Detections - {NETWORK}.{STATION}', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

print("✓ Example detections displayed")

## Detection Summary

In [ ]:
# Calculate high-confidence detections
earthquake_detections = (predictions == 2) & (probabilities[:, 2] > CONFIDENCE_THRESHOLD)
traffic_detections = (predictions == 1) & (probabilities[:, 1] > CONFIDENCE_THRESHOLD)

n_eq_detections = np.sum(earthquake_detections)
n_traffic_detections = np.sum(traffic_detections)

print("="*60)
print("DETECTION SUMMARY")
print("="*60)
print(f"\nStation: {NETWORK}.{STATION}.{CHANNEL}")
print(f"Time window: {trace.stats.starttime} to {trace.stats.endtime}")
print(f"Duration: {(trace.stats.endtime - trace.stats.starttime)/60:.1f} minutes")
print(f"\nTotal windows analyzed: {len(predictions)}")
print(f"Window size: {WINDOW_LENGTH_SEC:.1f} seconds")
print(f"Overlap: {WINDOW_OVERLAP*100:.0f}%")

print(f"\n--- All Predictions ---")
for label in range(num_classes):
    count = np.sum(predictions == label)
    percentage = count / len(predictions) * 100
    print(f"  {class_names[label]:<12}: {count:4d} windows ({percentage:5.1f}%)")

print(f"\n--- High Confidence Detections (>{CONFIDENCE_THRESHOLD*100:.0f}%) ---")
print(f"  Earthquakes: {n_eq_detections} windows")
print(f"  Traffic:     {n_traffic_detections} windows")

if n_eq_detections > 0:
    eq_times = times_sec[earthquake_detections]
    eq_confidences = probabilities[earthquake_detections, 2]
    print(f"\n--- Earthquake Detections ---")
    print(f"  Time range: {eq_times.min():.1f}s - {eq_times.max():.1f}s")
    print(f"  Peak confidence: {eq_confidences.max()*100:.1f}%")
    print(f"  Mean confidence: {eq_confidences.mean()*100:.1f}%")
    
    # Find continuous earthquake segments
    eq_indices = np.where(earthquake_detections)[0]
    gaps = np.diff(eq_indices)
    segment_starts = [eq_indices[0]]
    segment_ends = []
    
    for i, gap in enumerate(gaps):
        if gap > 2:  # Gap of more than 2 windows
            segment_ends.append(eq_indices[i])
            segment_starts.append(eq_indices[i+1])
    segment_ends.append(eq_indices[-1])
    
    print(f"\n  Detected {len(segment_starts)} earthquake segment(s):")
    for i, (start, end) in enumerate(zip(segment_starts, segment_ends)):
        duration = (times_sec[end] - times_sec[start]) + WINDOW_LENGTH_SEC
        print(f"    Segment {i+1}: {times_sec[start]:.1f}s - {times_sec[end]:.1f}s (duration: {duration:.1f}s)")

print("\n" + "="*60)

## Save Results (Optional)

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame({
    'window_id': np.arange(len(predictions)),
    'start_time': [str(t) for t in window_times],
    'time_sec': times_sec,
    'prediction': predictions,
    'predicted_class': [class_names[p] for p in predictions],
    'prob_noise': probabilities[:, 0],
    'prob_traffic': probabilities[:, 1],
    'prob_earthquake': probabilities[:, 2],
    'confidence': np.max(probabilities, axis=1)
})

# Save to CSV
output_dir = Path("predictions")
output_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"predictions_{STATION}_{timestamp}.csv"
results_df.to_csv(output_file, index=False)

print(f"✓ Results saved to: {output_file}")
print(f"  Total rows: {len(results_df)}")
print(f"\nFirst few predictions:")
print(results_df.head(10))